In [9]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

maze = np.array([
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 1],
    [1, 1, 1, 0, 1, 0, 1, 1, 0, 1],
    [1, 0, 0, 0, 0, 0, 1, 0, 0, 1],
    [1, 0, 1, 1, 1, 1, 1, 0, 1, 1],
    [1, 0, 1, 0, 0, 0, 0, 0, 1, 1],
    [1, 0, 1, 0, 1, 1, 1, 0, 1, 1],
    [1, 0, 1, 0, 1, 0, 0, 0, 1, 1],
    [1, 0, 1, 0, 1, 0, 1, 0, 0, 1],
    [1, 1, 1, 0, 1, 1, 1, 1, 0, 0]
])

start = (0, 0)
goal = (9, 9)

In [10]:
num_episodes = 5000 
alpha=0.1 
gamma=0.9
epsilon=0.5 

reward_fire=-10 
reward_goal=50 
reward_step=-1

actions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # right, down, left, up
Q=np.zeros(maze.shape+(len(actions),))

In [11]:
def is_valid_move(pos):
    r, c = pos
    if r < 0 or r >= maze.shape[0]:
        return False
    if c < 0 or c >= maze.shape[1]:
        return False
    if maze[r, c] == 1:
        return False
    return True

def choose_action(state):
    if np.random.rand() < epsilon:
        return np.random.choice(len(actions))
    return np.argmax(Q[state])


In [12]:
rewards_per_episodes = []
max_steps = 200

for episode in range(num_episodes):
    state = start
    total_reward = 0
    done = False

    for _ in range(max_steps):
        action_index = choose_action(state)
        action = actions[action_index]

        next_state = (state[0] + action[0], state[1] + action[1])

        if not is_valid_move(next_state):
            reward = reward_fire
            done = True
            next_max = 0
        else:
            if next_state == goal:
                reward = reward_goal
                done = True
            else:
                reward = reward_step
            next_max = np.max(Q[next_state])

        old_value = Q[state][action_index]
        Q[state][action_index] = old_value + alpha * (reward + gamma * next_max - old_value)

        state = next_state if is_valid_move(next_state) else state
        total_reward += reward

        if done:
            break

    epsilon = max(0.01, epsilon * 0.995)
    rewards_per_episodes.append(total_reward)


In [13]:
def get_optimal_path(Q, start, goal, actions, maze, max_steps=200):
    path = [start]
    state = start
    visited = set()

    for _ in range(max_steps):
        if state == goal:
            break
        visited.add(state)

        best_action = None
        for action_idx in np.argsort(Q[state])[::-1]:
            move = actions[action_idx]
            next_state = (state[0] + move[0], state[1] + move[1])

            if (0 <= next_state[0] < maze.shape[0] and
                0 <= next_state[1] < maze.shape[1] and
                maze[next_state] == 0 and
                next_state not in visited):
                best_action = action_idx
                break

        if best_action is None:
            break
        move = actions[best_action]
        state = (state[0] + move[0], state[1] + move[1])
        path.append(state)
    return path

optimal_path = get_optimal_path(Q, start, goal, actions, maze)


visualization 